<a href="https://colab.research.google.com/github/YoishD/Data-Science-B7/blob/main/Unit6/Unit6ExercisesSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 6 Exercises: Is my model good?

#### Over and Under fitting, Model Visualization, and Model/Variable Selection Concepts

These exercises are meant to get you to think about the model and variable selection process, and consider how we determine if a model is "good".

In [2]:
!pip install git+https://github.com/pydata/xarray.git
import xarray as xr
!pip install bambi
import bambi as bmb
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt

  Cloning https://github.com/pydata/xarray.git to /tmp/pip-req-build-otfvn1ol
  Running command git clone --filter=blob:none --quiet https://github.com/pydata/xarray.git /tmp/pip-req-build-otfvn1ol
  Resolved https://github.com/pydata/xarray.git to commit 1e5045a8c56a93d83d9347abcc020be2bb3cf622
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for xarray: filename=xarray-2025.1.1.dev4+g1e5045a8-py3-none-any.whl size=1241895 sha256=e080f3ba142c1fa43f6ec4f3d8621cafdc3e7bdff1c419c9c03e8f634ae3629c
  Stored in directory: /tmp/pip-ephem-wheel-cache-pw1fxk2_/wheels/3a/ec/c0/1cda725917279b6beba2a020d6ccddfee2176b25648f4fcad8
Successfully built xarray
  Attempting uninstall: xarray
    Found existing installation: xarray 2024.11.0
    Uninstalling xarray-2024.11.0:
      Successfully uninstalled xarray-2024.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 3.0 MB/s eta 0:00:00
  

**Task1**:

Does elpd_loo mean anything if we only have one model?

No Elpd_loo is based on comparason between two models

**Task2**:

Describe overfitting, in the context of this course

- selecting a model with the smallest number of variables ensures we are doing our best to avoid overfitting, or letting our model get distriacted by unneccesary information.


**Task3**:

How do we mitigate overfitting?

Priors, additonally elpd_loo helps pick a model that doesn't underfit too much

**Task4**:

How do we mitigate underfitting?

Selecting a model based on elpd_loo ensures we are doing our best to account for underfitting, or trying our best to be accurate.


**Task5**:

Why would we want more than one predictor in a model?

Having multiple predictors serve to better inform the model and thus lessen our assumptions, allowing our model to be more accurate.

**Task6**:

Can we have too many predictors? How would we now?

We definetly can have to many predictors and we can use elpd_loo and compare two models between each other to help visualize which model has the better number of predictors

**Task7**:

What is variable selection, and how does it work?

Variable selection is the process of figuring out the variables we need to add if any to become multiple predictors. In order to accomplish this you could run multipled different models favoring different variables and then comparing the results in order to better visualize the effects of the variables.

**Task8**:

Describe the differences and similarities between the following three models: linear regression with two predictors, one of which is a categorical variable:

- adding the variables in the model, as is standard.
- using that categorical variable as a hierarchy upon the other predictor variable.
- adding the variables, plus the categorical variable's interaction with the other variable.

**Task9**:

How do we visualize multiple linear regression models? Can we visualize the entire model, all at once?

**Task10**:

Compare the following linear models that all use the basketball data to predict field goal percentage:

- predictors free throw percentage and position (with position as a categorical predictor)
- predictors free throw percentage and position (with position as a hierarchy)
- predictors free throw percentage and position (with position interacting with frew throw percentage)
- predictors free throw percentage, position, 3 point attempts, and interactions between all three predictors
- predictors free throw percentage, position, 3 point attempts, with an interaction between 3 point attempts and postion.

using ```az.compare()``` and ```az.plot_compare()```, or an equivalent method using LOO (elpd_loo).

You may use the following two code blocks to load and clean the data.

In [3]:
#have to drop incomplete rows, so that bambi will run
bb = pd.read_csv(
    'https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/refs/heads/main/Data/basketball2324.csv').dropna()

In [4]:
#only look at players who played more than 600 minutes
#which is 20 min per game, for 30 games
bb = bb.query('MP > 600')
#remove players who never missed a free throw
bb = bb.query('`FT%` != 1.0')
#filter out the combo positions. This will make it easier to read the graphs
bb = bb.query("Pos in ['C','PF','SF','SG','PG']")
#gets rid of the annoying '%' sign
bb.rename(columns={"FT%":"FTp","FG%":"FGp"}, inplace=True)

<ipython-input-4-bb9f7b7913fd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb.rename(columns={"FT%":"FTp","FG%":"FGp"}, inplace=True)


In [7]:
priors ={"FTp" : bmb.Prior("Normal", mu=0,sigma=10), "alpha" : bmb.Prior("HalfNormal",sigma=10)}
model_nbb_bmb = bmb.Model("Pos ~ FTp", bb, family = "negativebinomial", priors=priors)

**Task11**:

Which model is "better" according to this metric?

Why do you think that is?